In [ ]:
import numpy as np
import pygrib
from scipy import interpolate
import multiprocessing as mp
import time

def linear_splines_unif(data, num_knots=10, level_width=1):   
    '''
    Calculates piecewise linear splines for quantile data using specified number of 
    knots uniformly spaced and returning interpolated approximation at every 
    quantile level with level_width.
    ''' 

    # dealing with all 0 data
    if data[-1] == 0:
        return np.zeros(int(np.floor(100/level_width)))

    # calculating where cdf starts being nonzero 
    levels = np.array(range(1,100))
    knot_ = np.where(data > 0)[0].min() - 1  

    # constructing knots
    if knot_ > 1:
        knots = np.unique(np.linspace(knot_-1, 98, num_knots-1, dtype=int))
        knots = np.insert(knots, 0, 0)
    else:
        knots = np.unique(np.linspace(0, 98, num_knots, dtype=int))
        
    # approx = interpolate.interp1d(knots+1, data[knots], assume_sorted=True) 
    # return approx(levels[level_width-1::level_width])
    return np.interp(levels[level_width-1::level_width], knots+1, data[knots])

# read in grib file
fn_grb = 'blend.t00z.qmd.f012.co.grib2'
ds_grb = pygrib.open(fn_grb)

# latitude and longitude grid values
lat, long = ds_grb.message(2).data()[1:]

# extracting precipitation levels for 6 hr forecast
precip_shape = lat.shape
precip_levels = np.zeros(shape=(99,)+precip_shape)
for i in range(99):
    precip_levels[i,:,:] = ds_grb.message(i+2).data()[0]

# flattening precip_levels
global precip_levels_flat
precip_levels_flat = np.zeros(shape=(99,lat.shape[0]*lat.shape[1]))

for element in range(lat.shape[0]*lat.shape[1]):
    i = int(np.floor(element/lat.shape[1]))
    j = int(element % lat.shape[1])
    precip_levels_flat[:,element] = precip_levels[:,i,j]

In [ ]:
nonzero_gridpoints = np.where(precip_levels_flat[-1,:] != 0)

In [ ]:
nonzero_gridpoints[0].shape

In [ ]:
lat.shape[0]*lat.shape[1] / np.where(precip_levels_flat[49,:] != 0)[0].shape[0] * 30/60